## Data Clean-up and Pre-Processing 

- Brian K
- Edited: 10/26/2019

In [1]:
import pandas as pd
import numpy as np
from collections import *
from typing import *
from string import punctuation
import spacy

# Custom Code imports
import sys
sys.path.insert(0, '/Users/briankalinowski/PycharmProjects/FakeNewsChallenge/Code')
import DataProcessing as pro

# All news


In [12]:
news_data = pd.read_csv('/Users/briankalinowski/Desktop/Data/General_News/all_the_news.csv')
news_data.drop(columns=['year', 'month'], inplace=True)
news_data.date = pd.to_datetime(news_data.date)
news_data.head()

,id,title,publication,author,date,url,content
0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,NaN,WASHINGTON — Congressional Republicans have...
1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,NaN,"After the bullet shells get counted, the blood..."
2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,NaN,"Death may be the great equalizer, but it isn’t..."
4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,NaN,"SEOUL, South Korea — North Korea’s leader, ..."


In [13]:
fill_vals = {'title': 'NO_TITLE', 
             'author': 'NO_AUTHOR', 
             'url': 'NO_URL'
            }

news_data = pro.fill_null_string_columns(news_data, fill_vals)

In [20]:
news_data.publication.value_counts()

Breitbart              23781
New York Post          17493
NPR                    11992
CNN                    11488
Washington Post        11114
Reuters                10710
Guardian                8681
New York Times          7803
Atlantic                7179
Business Insider        6757
National Review         6203
Talking Points Memo     5214
Vox                     4947
Buzzfeed News           4854
Fox News                4354
Name: publication, dtype: int64

In [17]:
news_content_len = news_data.content.str.split(' ').str.len()

In [19]:
news_content_len.mean()

779.0093427789858

In [9]:
news_content = news_data[['title', 'content', 'publication']]
news_content['content_length'] = news_content.content.str.len()
news_content = news_content.sort_values(by=['content_length'], ascending=False)
news_content= news_content.reset_index(drop=True)
news_content.head()

/anaconda/envs/FakeNewsChallenge/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,content,publication,content_length
0,Full transcript: FBI Director James Comey test...,Below is the full transcript from the...,Washington Post,292586
1,\nWill the Liberal Bubble Burst? Your Thoughts\n,Atlantic readers from across the political spe...,Atlantic,164658
2,Full transcript: Sally Yates and James Clapper...,Former acting attorney general Sally...,Washington Post,162120
3,\nTake It From a ‘Liberal Redneck’\n,Atlantic readers from across the political ...,Atlantic,162104
4,\nTrying to Avoid Politics Over Thanksgiving\n,Atlantic readers from across the political spe...,Atlantic,152708


In [10]:
news_content = pro.clean_text_column(news_content, 'title')
news_content = pro.clean_text_column(news_content, 'content')
news_content.head()

,title,content,publication,content_length
0,Full transcript FBI Director James Comey testi...,Below is the full transcript from the...,Washington Post,292586
1,Will the Liberal Bubble Burst? Your Thoughts,Atlantic readers from across the political spe...,Atlantic,164658
2,Full transcript Sally Yates and James Clapper ...,Former acting attorney general Sally...,Washington Post,162120
3,Take It From a Liberal Redneck,Atlantic readers from across the political ...,Atlantic,162104
4,Trying to Avoid Politics Over Thanksgiving,Atlantic readers from across the political spe...,Atlantic,152708


## Fake news 

In [2]:
fake_news = pd.read_csv('/Users/briankalinowski/Desktop/Data/Kaggle/fake_news.csv')
fake_news = fake_news[fake_news.language == 'english']
fake_news.drop(columns=['uuid', 'ord_in_thread', 'published'], inplace=True)
fake_news = fake_news.dropna(subset=['text'])
fake_news.head()

,author,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,Barracuda Brigade,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,reasoning with facts,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,Barracuda Brigade,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,Fed Up,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,Fed Up,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias


## Segment out just the text conetent for each article 

In [4]:
fake_news_content = fake_news[['title', 'text', 'type']]
fake_news_content = fake_news_content.reset_index(drop=True)
fake_news_content = pro.clean_text_column(fake_news_content, 'title')
fake_news_content = pro.clean_text_column(fake_news_content, 'text')

title_fill = {'title': 'notitle'}
fake_news_content = pro.fill_null_string_columns(fake_news_content, fill_dict=title_fill)
fake_news_content.head()

,title,text,type
0,Muslims BUSTED They Stole Millions In Govt Ben...,Print They should pay all the back all the mon...,bias
1,Re Why Did Attorney General Loretta Lynch Plea...,Why Did Attorney General Loretta Lynch Plead T...,bias
2,BREAKING Weiner Cooperating With FBI On Hillar...,Red State Fox News Sunday reported this mornin...,bias
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,bias
4,FANTASTIC! TRUMPS 7 POINT PLAN To Reform Healt...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,bias


In [5]:
fake_news_content.isnull().sum()

title    0
text     0
type     0
dtype: int64

In [6]:
fake_news_content.type.value_counts()

bs            10900
bias            443
conspiracy      430
hate            243
state           120
junksci         102
satire          100
fake             19
Name: type, dtype: int64

## Each article headline as a list by type 

In [7]:
# Each headline series as a list of text
fake_headlines = [str(hl) for hl in list(fake_news_content[fake_news_content.type == 'fake']['title'])]
bias_headlines = [str(hl) for hl in list(fake_news_content[fake_news_content.type == 'bias']['title'])]
consiracy_headlines = [str(hl) for hl in list(fake_news_content[fake_news_content.type == 'conspiracy']['title'])]
hate_headlines = [str(hl) for hl in list(fake_news_content[fake_news_content.type == 'hate']['title'])]
state_headlines = [str(hl) for hl in list(fake_news_content[fake_news_content.type == 'state']['title'])]
junksci_headlines = [str(hl) for hl in list(fake_news_content[fake_news_content.type == 'junksci']['title'])]
satire_headlines = [str(hl) for hl in list(fake_news_content[fake_news_content.type == 'satire']['title'])]
bs_headlines = [str(hl) for hl in list(fake_news_content[fake_news_content.type == 'bs']['title'])]

In [11]:
import spacy
from spacy import displacy

# Spacy English model
nlp = spacy.load('en_core_web_lg')

# Spacy procssing pipeline
spacy_pipline = nlp.pipe(fake_headlines)

# viz options
options = {"compact": True, "bg": "#09a3d5", "color": "white", 'arrow_stroke': 2}

for doc in spacy_pipline:
    displacy.render(doc, style="ent", jupyter=True, options=options) 
    displacy.render(doc, style="dep", jupyter=True, options=options)  

/Users/anaconda3/envs/FakeNewsChallenge/lib/python3.7/runpy.py:193: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  "__main__", mod_spec)


## Each article body as a list by type

In [111]:
# Each article body series as a list of text
fake_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'fake']['text'])]
bias_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'bias']['text'])]
consiracy_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'conspiracy']['text'])]
hate_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'hate']['text'])]
state_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'state']['text'])]
junksci_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'junksci']['text'])]
satire_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'satire']['text'])]
bs_content = [str(con) for con in list(fake_news_content[fake_news_content.type == 'bs']['text'])]

In [112]:
def nlp_text_clean(corpus):
    nlp = spacy.load('en_core_web_md')
    pipeline = nlp.pipe(corpus)
    ner_types = ['PERSON', 'NORP', 'ORG', 'GPE', 'EVENT', 'WORK_OF_ART', 'LAW', 'LANGUAGE']
    noun_types = ['PROPN', 'NOUN']

    for idx, doc in enumerate(pipeline):
        # Get all nouns and proper nouns
        noun_tokens = [token for token in doc if token.pos_ in noun_types]

        # Get entities from nouns
        entity_tokens = [token for token in noun_tokens if token.ent_type_ in ner_types]

        # Lemmatization of text (shortens it)
        lemma_tokens = [token.lemma_.strip().lower() for token in entity_tokens if token.is_stop is False
                        and token.text not in punctuation]
        
        if len(lemma_tokens) > 0:
            corpus[idx] = ' '.join(lemma_tokens)
        else:
            corpus[idx] = 'notokes'
    return corpus

In [113]:
title_array = pd.Series(fake_news_content.title).to_numpy(dtype=object, copy=True)
title_nlp = nlp_text_clean(title_array)    

In [114]:
fake_news_content.head()

,title,text,type
0,Muslims BUSTED They Stole Millions In Govt Ben...,Print They should pay all the back all the mon...,bias
1,Re Why Did Attorney General Loretta Lynch Plea...,Why Did Attorney General Loretta Lynch Plead T...,bias
2,BREAKING Weiner Cooperating With FBI On Hillar...,Red State Fox News Sunday reported this morni...,bias
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,bias
4,FANTASTIC! TRUMPS 7 POINT PLAN To Reform Healt...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,bias


In [115]:
fake_news_content['tokenized_headline'] = title_nlp

In [120]:
fake_news_content.head()

,title,text,type,tokenized_headline
0,Muslims BUSTED They Stole Millions In Govt Ben...,Print They should pay all the back all the mon...,bias,muslims
1,Re Why Did Attorney General Loretta Lynch Plea...,Why Did Attorney General Loretta Lynch Plead T...,bias,loretta lynch
2,BREAKING Weiner Cooperating With FBI On Hillar...,Red State Fox News Sunday reported this morni...,bias,weiner fbi hillary email investigation
3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,bias,donald j. trump
4,FANTASTIC! TRUMPS 7 POINT PLAN To Reform Healt...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,bias,malarky


In [121]:
text_array = pd.Series(fake_news_content.text).to_numpy(dtype=object, copy=True)
text_nlp = nlp_text_clean(text_array)  

In [123]:
fake_news_content['tokenized_content'] = text_nlp

In [128]:
space_test = fake_news.title.str.replace('[^0-9a-zA-Z .!?,]', '', regex=True).str.split().str.join(' ')

In [131]:
print(space_test[0:10])

0    Muslims BUSTED They Stole Millions In Govt Ben...
1    Re Why Did Attorney General Loretta Lynch Plea...
2    BREAKING Weiner Cooperating With FBI On Hillar...
3    PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...
4    FANTASTIC! TRUMPS 7 POINT PLAN To Reform Healt...
5    Hillary Goes Absolutely Berserk On Protester A...
6    BREAKING! NYPD Ready To Make Arrests In Weiner...
7    WOW! WHISTLEBLOWER TELLS CHILLING STORY Of Mas...
8    BREAKING CLINTON CLEARED...Was This A Coordina...
9    EVIL HILLARY SUPPORTERS Yell Fck TrumpBurn Tru...
Name: title, dtype: object


## NER Matrix

In [10]:
fake_ner = pro.run_ner_parse(bias_content)
fake_count_vec = pro.compute_doc_frequency_matrix(fake_ner)
fake_tfidf_vec = pro.compute_inverse_doc_matrix(fake_ner)

In [13]:
fake_count_vec

,000,100,100percentfedup,16,1990,2016,2nd,30,300k,302s,...,zadvydas,zealand,zeifman,zeke,zervos,zimbabwe,zoological,zora,zsl,zucker
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
399,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
400,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
fake_tfidf_vec

,000,100,100percentfedup,16,1990,2016,2nd,30,300k,302s,...,zadvydas,zealand,zeifman,zeke,zervos,zimbabwe,zoological,zora,zsl,zucker
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Proper Noun Matrix

In [15]:
fake_pro_nouns = pro.run_prop_noun_parse(bias_content)
fake_pro_count_vec = pro.compute_doc_frequency_matrix(fake_pro_nouns)
fake_pro_tfidf_vec = pro.compute_inverse_doc_matrix(fake_pro_nouns)

In [16]:
fake_pro_count_vec

,14th,1st,2014media,2020even,263room,2nd,302s,3a6,455a,501c3,...,zeifman,zeke,zervos,zimbabwe,zone,zones,zoological,zora,zsl,zucker
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
415,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
416,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
fake_pro_tfidf_vec

,14th,1st,2014media,2020even,263room,2nd,302s,3a6,455a,501c3,...,zeifman,zeke,zervos,zimbabwe,zone,zones,zoological,zora,zsl,zucker
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
414,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
415,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Spacy Viz Testing

In [29]:
import spacy
from spacy import displacy

#test = fake_headlines[18]
test = 'Lady Gagas Twitter Attack On Melania Trump Lands Her In Handcuffs When The Two Meet Face To Face  The Resistance The Last Line of Defense'
nlp = spacy.load('en_core_web_md')
test_doc = nlp(test)

options = {"compact": True, 
           "bg": "#09a3d5",
           "color": "white", 
           "font": "Source Sans Pro"}

displacy.render(test_doc, style="dep", jupyter=True, options=options)
displacy.render(test_doc, style="ent", jupyter=True, options=options)

In [11]:
import spacy
nlp = spacy.load('en')

doc = nlp(fake_headlines[18])

print('Raw Text:', fake_headlines[18], '\n')
print('Lemmatized Text:',' '.join([token.lemma_ for token in doc]))

Raw Text: Lady Gagas Twitter Attack On Melania Trump Lands Her In Handcuffs When The Two Meet Face To Face  The Resistance The Last Line of Defense 

Lemmatized Text: Lady Gagas Twitter Attack on Melania Trump land -PRON- in handcuff when the two Meet face to face   the Resistance the last line of Defense


In [23]:
from spacy.lang.en.stop_words import STOP_WORDS
print(list(STOP_WORDS))

['throughout', 'either', 'hundred', 'herself', 'us', 'once', 'rather', 'several', 'very', 'do', 'top', 'neither', 'each', 'since', 'nine', 'were', 'among', 'being', 'in', 'more', 'various', 'cannot', 'hereupon', 'when', 'is', 'nothing', 'ever', 'only', 'due', 'into', 'seemed', 'over', '‘ll', 'between', 'whenever', 'even', 'however', 'have', 'must', 'along', 'mostly', 'such', 'else', 'their', 'herein', 'though', 'bottom', 'became', 'yet', 'why', 'within', 'about', 'hereby', 'i', 'alone', 'his', 'never', 'off', 'amongst', 'to', 'say', 'all', 'full', 'then', 'none', 'some', 'whither', 'first', 'thereby', 'whether', '’m', 'eleven', 'take', 'thereafter', 'but', 'becoming', 'something', "'re", 'she', 'we', 'quite', 'toward', 'himself', 'using', 'indeed', 'hereafter', 'two', 'own', 'yourselves', 'should', 'yours', 'below', 'could', 'those', 'hence', 'whom', 'anything', 'perhaps', 'eight', 'at', 'an', 'before', 'often', 'wherever', 'just', "'m", 'another', 'he', 'fifty', 'thru', 'they', 'any',

In [25]:
print('We' in STOP_WORDS)

False
